In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path="../../models/hf"
model_path="../../models/llama/hf"
model_path="../../models/llama/conv_hf"
model_path="../../models/hf/1code"
model_path="../../models/hf/2jaA"
model_path="../../models/hf/step34300"
#model_path= "llm-jp/llm-jp-13b-v1.0"
#model_path="llm-jp/llm-jp-13b-v1.0"
#model_path="matsuo-lab/weblab-10b"

In [3]:

model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")

In [ ]:
#for name, param in model.named_parameters():
#    print(name)

In [4]:

tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
from tokenizers import AddedToken
tokenizer.add_tokens(AddedToken("\n", normalized=False))
t="\n\n### 指示:\n"
#t=":### 指示::"
#t="<SEP>指示<SEP>"
t_list=[
"\n\n### 指示:\n",
"」\n\n### 指示:\n",
"ああ\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nこんにちは",
"\n\n### 指示:\nThis",
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nこんにちは",
"\n\n### 応答:\nThis",

]
for t in t_list:
    print(tokenizer.encode(t),tokenizer.tokenize(t))

[8, 8, 5092, 272, 3994, 327, 8, 2] ['\n', '\n', '▁###', '▁', '指示', ':', '\n']
[272, 285, 8, 8, 5092, 272, 3994, 327, 8, 2] ['▁', '」', '\n', '\n', '▁###', '▁', '指示', ':', '\n']
[8722, 8, 8, 5092, 272, 3994, 327, 8, 2] ['▁ああ', '\n', '\n', '▁###', '▁', '指示', ':', '\n']
[8, 8, 5092, 272, 3994, 327, 8, 337, 2] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁「']
[8, 8, 5092, 272, 3994, 327, 8, 272, 3124, 2] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'こんにちは']
[8, 8, 5092, 272, 3994, 327, 8, 1417, 2] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁This']
[8, 8, 5092, 272, 1045, 2850, 327, 8, 2] ['\n', '\n', '▁###', '▁', '応', '答', ':', '\n']
[8, 8, 5092, 272, 1045, 2850, 327, 8, 337, 2] ['\n', '\n', '▁###', '▁', '応', '答', ':', '\n', '▁「']
[8, 8, 5092, 272, 1045, 2850, 327, 8, 272, 3124, 2] ['\n', '\n', '▁###', '▁', '応', '答', ':', '\n', '▁', 'こんにちは']
[8, 8, 5092, 272, 1045, 2850, 327, 8, 1417, 2] ['\n', '\n', '▁###', '▁', '応', '答', ':', '\n', '▁This']


In [11]:

tokenizer = AutoTokenizer.from_pretrained(model_path)
print(type(tokenizer))
tokenizer.add_tokens(AddedToken("\n", normalized=False))
a="ああ\nああ"
tok=tokenizer.encode(a)
back=tokenizer.decode(tok)
a,tok,back

<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


('ああ\nああ', [8722, 8, 8722, 2], 'ああ\n ああ</s>')

In [ ]:
t="[###指示]ブラジル"
t="[###指示]元気ですか?"
t="###指示:ブラジル"
t="###指示:東京"
t="###応答:東京"
t="きなさい。:### 指示::元気ブラジル"
t="きなさい。:### 指示::次の"
#t="[###応答]「中古"
#t="<SEP> 指示<SEP> この「オーロラ作戦」の記事を踏まえて、攻撃の対象となった企業はどこでしょうか? オーロラ作戦は、"
#t="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP> 指示<SEP> この「オーロラ作戦」の記事を踏まえて、攻撃の対象となった企業はどこでしょうか? "
tokenizer.encode(t),tokenizer.tokenize(t)

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=200, 
              repetition_penalty=1.2,
              temperature=0.7,
              )

In [ ]:
import time
text_list=[
]


text_list=[
"llamaのconvertは大変" ,   
"今日はいい",
"富士山は",
"質問: 今日の天気は? 回答:",
"批判的な",
"大規模言語モデルは",
"AI研究の問題点は",
"化学研究の問題点は",
"I have a",
"Functional polymers are",
"機能性高分子は",           
"ホンダ フィット 販売",
"英語: He is a good man. 日本語: ",
"Q: 日本の首都は? A:",
"Q: ドラえもんの友達は? A:",
"Q: ドラえもんとはなんですか? A:",
"Q: 子育ての秘訣は? A:",
"Q: 純粋理性批判はたぬきに理解できますか? A:",
]

for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("input: ", text)
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)